# Basic Data Exploration for BA Consult Group Project

Aims:
1. Combine and clean the three datasets I found into a single one for regression
2. Decide if to use a single region with year (30 rows max), or multiple regions (30 * 3 regions = 90 rows), which is better for regressions
    - Limitations due to different sampling methods, recent years uses a newer one
    - Lack of data, especially for SEA region waters + harder to split
    -

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1.1 Dataset 1 - Microplastic Concentration
Consist of 3 Portions:
1. Aggregate based on a single region and year
2. Aggregate based on a single region and year-month
3. Aggregate based on multiple regions and year

Chosen Regions:
1. US East Coast
2. Gulf of Mexico
3. US West Coast

#### 1.1.1 Aggregate based on a Single Region and (Year, Sampling Method)s

In [ ]:
# Specify the 3 regions coordinates
REGIONS = {
    "US_EAST_COAST": {
        "lat_min": 25,
        "lat_max": 47,
        "lon_min": -82,
        "lon_max": -66      
    },
    "GULF_OF_MEXICO": {
        "lat_min": 18,
        "lat_max": 31,
        "lon_min": -98,
        "lon_max": -80
    },
    "US_WEST_COAST": {
        # Bounding Box for US West 
        "lat_min": 32, 
        "lat_max": 49,
        "lon_min": -130, 
        "lon_max": -117
    },
}

# Load the Microplastic Dataset into a DF
microplastic_df = pd.read_csv("./datasets/marine_microplastic_ocean_global.csv")
microplastic_df.head()

,OBJECTID,Latitude (degree),Longitude (degree),Ocean,Region,Subregion,Country,State,Beach Location,Marine Setting,...,DOI,Organization,Keywords,NCEI Accession Number,NCEI Accession Link,Symbology,GlobalID,Sample Date,x,y
0,1,45.28,-60.29,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Very Low, 0-0.0005 pieces/m3",2ce1d1d4-4231-4ab8-af74-0b7fbb1e028b,7/13/1989 12:00:00 AM,-60.29,45.28
1,2,40.93,-70.65,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Low, 0.0005-0.005 pieces/m3",8e3c8426-5ab0-421e-abb2-6b90197ada7a,10/12/1989 12:00:00 AM,-70.65,40.93
2,3,40.93,-70.65,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Low, 0.0005-0.005 pieces/m3",99bda337-9f0c-4f58-9960-8b00c1110a7c,10/12/1989 12:00:00 AM,-70.65,40.93
3,4,40.30,-69.77,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Very Low, 0-0.0005 pieces/m3",1f20e926-2a84-4e42-8478-8a5af5aecacc,10/12/1989 12:00:00 AM,-69.77,40.30
4,5,39.88,-67.15,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,Ocean water,...,https://doi.org/10.1126/science.1192321,Sea Education Association,SEA,211007,https://www.ncei.noaa.gov/access/metadata/land...,"Ocean Surface, Very Low, 0-0.0005 pieces/m3",a0df6bee-9c60-4621-a0bc-3329e3f11720,10/15/1989 12:00:00 AM,-67.15,39.88


In [3]:
# Convert all the sample dates into Datetime objects
microplastic_df["Datetime"] = pd.to_datetime(
    microplastic_df["Sample Date"],
    errors="coerce"
)

# Extract the Year and Month as separate columns
microplastic_df["Year"] = microplastic_df["Datetime"].dt.year
# Create YearMonth column
microplastic_df["YearMonth"] = microplastic_df["Datetime"].dt.to_period("M")

# Print to seee
print("Unique YearMonth values:", microplastic_df["YearMonth"].nunique())
print("Min/Max:", microplastic_df["YearMonth"].min(), microplastic_df["YearMonth"].max())

/var/folders/w1/mvky9f4d4s19c7tj7wqmqyx00000gn/T/ipykernel_48532/896251586.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  microplastic_df["Datetime"] = pd.to_datetime(


Unique YearMonth values: 396
Min/Max: 1972-04 2023-02


In [4]:
# Format the sampling method correctly
microplastic_df["Sampling Method"] = (
    microplastic_df["Sampling Method"]
    .astype(str)
    .str.strip()
)

# Set the Microplastic Concentration correctly
microplastic_df["Microplastic_Concentration"] = pd.to_numeric(
    microplastic_df["Microplastics Measurement"],
    errors="coerce"
)

**1.US East Coast**

In [5]:
# Bounding Box for US East Coast
SELECTED_REGION = "US_EAST_COAST"
lat_min = REGIONS[SELECTED_REGION]["lat_min"]
lat_max = REGIONS[SELECTED_REGION]["lat_max"]
lon_min = REGIONS[SELECTED_REGION]["lon_min"]
lon_max = REGIONS[SELECTED_REGION]["lon_max"]

# Filter out the rows in the US East Coast
microplastic_df["Latitude (degree)"] = pd.to_numeric(microplastic_df["Latitude (degree)"], errors="coerce")
microplastic_df["Longitude (degree)"] = pd.to_numeric(microplastic_df["Longitude (degree)"], errors="coerce")

# Cretate a new df for east coast
us_east_df = microplastic_df[
    microplastic_df["Latitude (degree)"].between(lat_min, lat_max) &
    microplastic_df["Longitude (degree)"].between(lon_min, lon_max)
].copy()

# Drop missing values if they are missing year, sampling method and conc since we cant use
us_east_df = us_east_df.dropna(subset=["Year", "Sampling Method", "Microplastic_Concentration"])

# Aggregate based on (Year, Sampling Method) tuple
cleaned_us_east_df = (
    us_east_df
    .groupby(["Year", "Sampling Method"], as_index=False)
    .agg(
        Microplastic_Mean=("Microplastic_Concentration", "mean"),
        Microplastic_Median=("Microplastic_Concentration", "median"),
        Total_Samples=("Microplastic_Concentration", "size")
    )
    .sort_values(["Year", "Sampling Method"])
)


print(f"Results Summary (US East; 1 Region; (Year, Sampling Method) Aggregate)")
print("Number of microplastic_df Rows: ", len(microplastic_df))
print("Number of us_east_df Rows: ", len(us_east_df))
print("Number of cleaned_us_east_df Rows: ", len(cleaned_us_east_df))
cleaned_us_east_df.head()

Results Summary (US East; 1 Region; (Year, Sampling Method) Aggregate)
Number of microplastic_df Rows:  22530
Number of us_east_df Rows:  2263
Number of cleaned_us_east_df Rows:  32


,Year,Sampling Method,Microplastic_Mean,Microplastic_Median,Total_Samples
0,1986,Neuston net,0.000000,0.00000,2
1,1987,Neuston net,0.019094,0.01080,37
2,1989,Neuston net,0.021676,0.00648,36
3,1990,Neuston net,0.046638,0.00972,38
4,1991,Neuston net,0.036560,0.01296,81


**2. Gulf of Mexico**

In [6]:
# Bounding Box for GULF_OF_MEXICO
SELECTED_REGION = "GULF_OF_MEXICO"
lat_min = REGIONS[SELECTED_REGION]["lat_min"]
lat_max = REGIONS[SELECTED_REGION]["lat_max"]
lon_min = REGIONS[SELECTED_REGION]["lon_min"]
lon_max = REGIONS[SELECTED_REGION]["lon_max"]

# Filter out the rows in the GULF_OF_MEXICO
microplastic_df["Latitude (degree)"] = pd.to_numeric(microplastic_df["Latitude (degree)"], errors="coerce")
microplastic_df["Longitude (degree)"] = pd.to_numeric(microplastic_df["Longitude (degree)"], errors="coerce")

# Cretate a new df for GULF_OF_MEXICO
gulf_of_mexico_df = microplastic_df[
    microplastic_df["Latitude (degree)"].between(lat_min, lat_max) &
    microplastic_df["Longitude (degree)"].between(lon_min, lon_max)
].copy()

# Drop missing values if they are missing year, sampling method and conc since we cant use
gulf_of_mexico_df = gulf_of_mexico_df.dropna(subset=["Year", "Sampling Method", "Microplastic_Concentration"])

# Aggregate based on (Year, Sampling Method) tuple
cleaned_gulf_of_mexico_df = (
    gulf_of_mexico_df
    .groupby(["Year", "Sampling Method"], as_index=False)
    .agg(
        Microplastic_Mean=("Microplastic_Concentration", "mean"),
        Microplastic_Median=("Microplastic_Concentration", "median"),
        Total_Samples=("Microplastic_Concentration", "size")
    )
    .sort_values(["Year", "Sampling Method"])
)


print(f"Results Summary (GULF_OF_MEXICO; 1 Region; (Year, Sampling Method) Aggregate)")
print("Number of microplastic_df Rows: ", len(microplastic_df))
print("Number of gulf_of_mexico_df Rows: ", len(gulf_of_mexico_df))
print("Number of cleaned_gulf_of_mexico_df Rows: ", len(cleaned_gulf_of_mexico_df))

Results Summary (GULF_OF_MEXICO; 1 Region; (Year, Sampling Method) Aggregate)
Number of microplastic_df Rows:  22530
Number of gulf_of_mexico_df Rows:  769
Number of cleaned_gulf_of_mexico_df Rows:  28


**3.US West Coast**

In [7]:
# Bounding Box for US West Coast
SELECTED_REGION = "US_WEST_COAST"
lat_min = REGIONS[SELECTED_REGION]["lat_min"]
lat_max = REGIONS[SELECTED_REGION]["lat_max"]
lon_min = REGIONS[SELECTED_REGION]["lon_min"]
lon_max = REGIONS[SELECTED_REGION]["lon_max"]

# Filter out the rows in the US West Coast
microplastic_df["Latitude (degree)"] = pd.to_numeric(microplastic_df["Latitude (degree)"], errors="coerce")
microplastic_df["Longitude (degree)"] = pd.to_numeric(microplastic_df["Longitude (degree)"], errors="coerce")

# Cretate a new df for west coast
us_west_df = microplastic_df[
    microplastic_df["Latitude (degree)"].between(lat_min, lat_max) &
    microplastic_df["Longitude (degree)"].between(lon_min, lon_max)
].copy()

# Drop missing values if they are missing year, sampling method and conc since we cant use
us_west_df = us_west_df.dropna(subset=["Year", "Sampling Method", "Microplastic_Concentration"])

# Aggregate based on (Year, Sampling Method) tuple
cleaned_us_west_df = (
    us_west_df
    .groupby(["Year", "Sampling Method"], as_index=False)
    .agg(
        Microplastic_Mean=("Microplastic_Concentration", "mean"),
        Microplastic_Median=("Microplastic_Concentration", "median"),
        Total_Samples=("Microplastic_Concentration", "size")
    )
    .sort_values(["Year", "Sampling Method"])
)


print(f"Results Summary (US West; 1 Region; (Year, Sampling Method) Aggregate)")
print("Number of microplastic_df Rows: ", len(microplastic_df))
print("Number of us_west_df Rows: ", len(us_west_df))
print("Number of cleaned_us_west_df Rows: ", len(cleaned_us_west_df))

Results Summary (US West; 1 Region; (Year, Sampling Method) Aggregate)
Number of microplastic_df Rows:  22530
Number of us_west_df Rows:  182
Number of cleaned_us_west_df Rows:  21


#### 1.1.2 Aggregate based on a Single Region and (MonthYear, Sampling Method)

In [8]:
# Bounding Box for US East Coast
SELECTED_REGION = "US_EAST_COAST"
lat_min = REGIONS[SELECTED_REGION]["lat_min"]
lat_max = REGIONS[SELECTED_REGION]["lat_max"]
lon_min = REGIONS[SELECTED_REGION]["lon_min"]
lon_max = REGIONS[SELECTED_REGION]["lon_max"]

# Filter out rows in US_EAST_COAST
microplastic_df["Latitude (degree)"] = pd.to_numeric(microplastic_df["Latitude (degree)"], errors="coerce")
microplastic_df["Longitude (degree)"] = pd.to_numeric(microplastic_df["Longitude (degree)"], errors="coerce")

us_east_df = microplastic_df[
    microplastic_df["Latitude (degree)"].between(lat_min, lat_max) &
    microplastic_df["Longitude (degree)"].between(lon_min, lon_max)
].copy()

# Drop missing core values
us_east_df = us_east_df.dropna(subset=["Year", "Sampling Method", "Microplastic_Concentration"])

# Aggregate based on (Year, Sampling Method) tuple
cleaned_us_east_df = (
    us_east_df
    .groupby(["YearMonth", "Sampling Method"], as_index=False)
    .agg(
        Microplastic_Mean=("Microplastic_Concentration", "mean"),
        Microplastic_Median=("Microplastic_Concentration", "median"),
        Total_Samples=("Microplastic_Concentration", "size")
    )
    .sort_values(["YearMonth", "Sampling Method"])
)


print(f"Results Summary (US East; 1 Region; (YearMonth, Sampling Method) Aggregate)")
print("Number of microplastic_df Rows: ", len(microplastic_df))
print("Number of us_east_df Rows: ", len(us_east_df))
print("Number of cleaned_us_east_df Rows: ", len(cleaned_us_east_df))


Results Summary (US East; 1 Region; (YearMonth, Sampling Method) Aggregate)
Number of microplastic_df Rows:  22530
Number of us_east_df Rows:  2263
Number of cleaned_us_east_df Rows:  199


In [9]:
# Bounding Box for GULF_OF_MEXICO
SELECTED_REGION = "GULF_OF_MEXICO"
lat_min = REGIONS[SELECTED_REGION]["lat_min"]
lat_max = REGIONS[SELECTED_REGION]["lat_max"]
lon_min = REGIONS[SELECTED_REGION]["lon_min"]
lon_max = REGIONS[SELECTED_REGION]["lon_max"]

# Filter out rows in GULF_OF_MEXICO
microplastic_df["Latitude (degree)"] = pd.to_numeric(microplastic_df["Latitude (degree)"], errors="coerce")
microplastic_df["Longitude (degree)"] = pd.to_numeric(microplastic_df["Longitude (degree)"], errors="coerce")

gulf_of_mexico_df = microplastic_df[
    microplastic_df["Latitude (degree)"].between(lat_min, lat_max) &
    microplastic_df["Longitude (degree)"].between(lon_min, lon_max)
].copy()

# Drop missing core values
gulf_of_mexico_df = gulf_of_mexico_df.dropna(subset=["Year", "Sampling Method", "Microplastic_Concentration"])

# Aggregate based on (Year, Sampling Method) tuple
cleaned_gulf_of_mexico_df = (
    gulf_of_mexico_df
    .groupby(["YearMonth", "Sampling Method"], as_index=False)
    .agg(
        Microplastic_Mean=("Microplastic_Concentration", "mean"),
        Microplastic_Median=("Microplastic_Concentration", "median"),
        Total_Samples=("Microplastic_Concentration", "size")
    )
    .sort_values(["YearMonth", "Sampling Method"])
)


print(f"Results Summary (GULF_OF_MEXICO; 1 Region; (YearMonth, Sampling Method) Aggregate)")
print("Number of microplastic_df Rows: ", len(microplastic_df))
print("Number of gulf_of_mexico_df Rows: ", len(gulf_of_mexico_df))
print("Number of cleaned_gulf_of_mexico_df Rows: ", len(cleaned_gulf_of_mexico_df))


Results Summary (GULF_OF_MEXICO; 1 Region; (YearMonth, Sampling Method) Aggregate)
Number of microplastic_df Rows:  22530
Number of gulf_of_mexico_df Rows:  769
Number of cleaned_gulf_of_mexico_df Rows:  101


In [10]:
# Bounding Box for US West Coast
SELECTED_REGION = "US_WEST_COAST"
lat_min = REGIONS[SELECTED_REGION]["lat_min"]
lat_max = REGIONS[SELECTED_REGION]["lat_max"]
lon_min = REGIONS[SELECTED_REGION]["lon_min"]
lon_max = REGIONS[SELECTED_REGION]["lon_max"]

# Filter out rows in US_WEST_COAST
microplastic_df["Latitude (degree)"] = pd.to_numeric(microplastic_df["Latitude (degree)"], errors="coerce")
microplastic_df["Longitude (degree)"] = pd.to_numeric(microplastic_df["Longitude (degree)"], errors="coerce")

us_west_df = microplastic_df[
    microplastic_df["Latitude (degree)"].between(lat_min, lat_max) &
    microplastic_df["Longitude (degree)"].between(lon_min, lon_max)
].copy()

# Drop missing core values
us_west_df = us_west_df.dropna(subset=["Year", "Sampling Method", "Microplastic_Concentration"])

# Aggregate based on (Year, Sampling Method) tuple
cleaned_us_west_df = (
    us_west_df
    .groupby(["YearMonth", "Sampling Method"], as_index=False)
    .agg(
        Microplastic_Mean=("Microplastic_Concentration", "mean"),
        Microplastic_Median=("Microplastic_Concentration", "median"),
        Total_Samples=("Microplastic_Concentration", "size")
    )
    .sort_values(["YearMonth", "Sampling Method"])
)


print(f"Results Summary (US West; 1 Region; (YearMonth, Sampling Method) Aggregate)")
print("Number of microplastic_df Rows: ", len(microplastic_df))
print("Number of us_west_df Rows: ", len(us_west_df))
print("Number of cleaned_us_west_df Rows: ", len(cleaned_us_west_df))

Results Summary (US West; 1 Region; (YearMonth, Sampling Method) Aggregate)
Number of microplastic_df Rows:  22530
Number of us_west_df Rows:  182
Number of cleaned_us_west_df Rows:  52


#### 1.1.3 Aggregate based on all regions in the US and (Year, Sampling Method)

In [11]:
# Combine multiple US Regions since we are working with Annual Fish Production
SELECTED_REGIONS = ["US_EAST_COAST", "US_WEST_COAST", "GULF_OF_MEXICO"]

region_dfs = []
for region_name in SELECTED_REGIONS:
    # Filter the rows in the selected region
    bounds = REGIONS[region_name]
    region_df = microplastic_df[
        microplastic_df["Latitude (degree)"].between(bounds["lat_min"], bounds["lat_max"]) &
        microplastic_df["Longitude (degree)"].between(bounds["lon_min"], bounds["lon_max"])
    ].copy()
    region_df["Region"] = region_name

    # Drop missing if missing any of the 3 values
    region_df = region_df.dropna(subset=["Year", "Sampling Method", "Microplastic_Concentration"])

    # Aggregate within this region
    cleaned_region_df = (
        region_df
        .groupby(["Region", "Year", "Sampling Method"], as_index=False)
        .agg(
            Microplastic_Mean=("Microplastic_Concentration", "mean"),
            Microplastic_Median=("Microplastic_Concentration", "median"),
            Total_Samples=("Microplastic_Concentration", "size"),
        )
        .sort_values(["Region", "Year", "Sampling Method"])
    )

    # Add to array of dfs
    region_dfs.append(cleaned_region_df)

# Combined all df together
cleaned_us_all_regions_df = pd.concat(region_dfs, ignore_index=True)

print("Results Summary (US East + West + Gulf; (Region, Year, Sampling Method) Aggregate)")
print("Number of microplastic_df Rows:", len(microplastic_df))
print("Number of cleaned_us_all_regions_df Rows:", len(cleaned_us_all_regions_df))
print(cleaned_us_all_regions_df.head())

Results Summary (US East + West + Gulf; (Region, Year, Sampling Method) Aggregate)
Number of microplastic_df Rows: 22530
Number of cleaned_us_all_regions_df Rows: 81
          Region  Year Sampling Method  Microplastic_Mean  \
0  US_EAST_COAST  1986     Neuston net           0.000000   
1  US_EAST_COAST  1987     Neuston net           0.019094   
2  US_EAST_COAST  1989     Neuston net           0.021676   
3  US_EAST_COAST  1990     Neuston net           0.046638   
4  US_EAST_COAST  1991     Neuston net           0.036560   

   Microplastic_Median  Total_Samples  
0              0.00000              2  
1              0.01080             37  
2              0.00648             36  
3              0.00972             38  
4              0.01296             81  


### 1.2 Dataset 2 - Aquaculture Production

- Can use fisheries production as a proxy for aquaculture

In [28]:
aquaculture_df = pd.read_csv("./datasets/us_three_regions_fish_production_annual.csv")

aquaculture_df.head()

,Year,Region Name,Pounds,Dollars,Collection,Metric Tons
0,1950,Gulf,"570,641,100","50,357,945",Commercial,"258,841"
1,1950,Middle Atlantic,"873,150,700","53,973,253",Commercial,"396,059"
2,1950,New England,"1,006,589,500","60,576,490",Commercial,"456,586"
3,1950,Pacific Coast,"1,716,453,400","110,917,048",Commercial,"778,578"
4,1950,South Atlantic,"260,881,400","18,943,779",Commercial,"118,335"


In [ ]:
# Clean column names
aquaculture_df.columns = [c.strip().lower().replace(" ", "_") for c in aquaculture_df.columns]

# Cast to Numeric
def convertToNumber(series):
    return (
        series.astype(str)
        .str.replace(",", "", regex=False)
        .str.replace("$", "", regex=False)
        .str.strip()
        .replace({"": np.nan, "nan": np.nan})
        .astype(float)
    )
aquaculture_df["year"] = pd.to_numeric(aquaculture_df["year"], errors="coerce").astype("Int64")
aquaculture_df["pounds"] = convertToNumber(aquaculture_df["pounds"])
aquaculture_df["dollars"] = convertToNumber(aquaculture_df["dollars"])
aquaculture_df["metric_tons"] = convertToNumber(aquaculture_df["metric_tons"])

# Assign to 1 of the 3 Regions
REGION_MAP = {
    "Gulf": "GULF_OF_MEXICO",
    "Pacific Coast": "US_WEST_COAST",
    "New England": "US_EAST_COAST",
    "Middle Atlantic": "US_EAST_COAST",
    "South Atlantic": "US_EAST_COAST",
    # Not in any of hte 3 regions
    "Not Applicable to Three Regions": np.nan,
}

aquaculture_df["assigned_region"] = aquaculture_df["region_name"].map(REGION_MAP)
aquaculture_df.head(n=10)

,year,region_name,pounds,dollars,collection,metric_tons,assigned_region
0,1950,Gulf,5.706411e+08,50357945.0,Commercial,258841.0,GULF_OF_MEXICO
1,1950,Middle Atlantic,8.731507e+08,53973253.0,Commercial,396059.0,US_EAST_COAST
2,1950,New England,1.006590e+09,60576490.0,Commercial,456586.0,US_EAST_COAST
3,1950,Pacific Coast,1.716453e+09,110917048.0,Commercial,778578.0,US_WEST_COAST
4,1950,South Atlantic,2.608814e+08,18943779.0,Commercial,118335.0,US_EAST_COAST
5,1951,Gulf,6.496683e+08,63623145.0,Commercial,294688.0,GULF_OF_MEXICO
6,1951,Middle Atlantic,8.867815e+08,54741099.0,Commercial,402241.0,US_EAST_COAST
7,1951,New England,9.168076e+08,67431668.0,Commercial,415861.0,US_EAST_COAST
8,1951,Pacific Coast,1.296517e+09,100746805.0,Commercial,588096.0,US_WEST_COAST
9,1951,South Atlantic,3.107793e+08,17965732.0,Commercial,140969.0,US_EAST_COAST


In [30]:
# Extract only the relevant fields
aquaculture_df = aquaculture_df[["year", "metric_tons", "dollars", "assigned_region"]].copy()

# Keep rows in the 3 regions only
print("Number of Rows Before Drop: ", len(aquaculture_df))
aquaculture_df = aquaculture_df.dropna(subset=["assigned_region"])
print("Number of Rows After Drop: ", len(aquaculture_df))

# Sort based on year
aquaculture_df = aquaculture_df.sort_values(["assigned_region", "year"]).reset_index(drop=True)
aquaculture_df["year"] = aquaculture_df["year"].astype("Int64")

aquaculture_df.head(10)

Number of Rows Before Drop:  641
Number of Rows After Drop:  593


,year,metric_tons,dollars,assigned_region
0,1950,258841.0,50357945.0,GULF_OF_MEXICO
1,1951,294688.0,63623145.0,GULF_OF_MEXICO
2,1952,344510.0,68220786.0,GULF_OF_MEXICO
3,1953,344458.0,85157157.0,GULF_OF_MEXICO
4,1954,342330.0,72572237.0,GULF_OF_MEXICO
5,1955,376322.0,76286847.0,GULF_OF_MEXICO
6,1956,408135.0,85377584.0,GULF_OF_MEXICO
7,1957,313859.0,84363714.0,GULF_OF_MEXICO
8,1958,366886.0,86539158.0,GULF_OF_MEXICO
9,1959,523882.0,77639851.0,GULF_OF_MEXICO


**Join the Aquaculture dataset with the Microplastic**

In [31]:
# Create a copy of the microplastic cleaned datasets for all 3 regions
cleaned_microplastic_df = cleaned_us_all_regions_df.copy()
cleaned_microplastic_df = cleaned_microplastic_df.rename(columns={"Year": "year", "Region": "assigned_region"})
cleaned_microplastic_df["year"] = pd.to_numeric(cleaned_microplastic_df["year"], errors="coerce").astype("Int64")

# JOIN with microplastic dataset
merged_microplastic_aqua_df = aquaculture_df.merge(
    cleaned_microplastic_df,
    on=["year", "assigned_region"],
    how="left"
)
print("Number of rows", len(merged_microplastic_aqua_df))
merged_microplastic_aqua_df.head()

Number of rows 625


,year,metric_tons,dollars,assigned_region,Sampling Method,Microplastic_Mean,Microplastic_Median,Total_Samples
0,1950,258841.0,50357945.0,GULF_OF_MEXICO,NaN,NaN,NaN,NaN
1,1951,294688.0,63623145.0,GULF_OF_MEXICO,NaN,NaN,NaN,NaN
2,1952,344510.0,68220786.0,GULF_OF_MEXICO,NaN,NaN,NaN,NaN
3,1953,344458.0,85157157.0,GULF_OF_MEXICO,NaN,NaN,NaN,NaN
4,1954,342330.0,72572237.0,GULF_OF_MEXICO,NaN,NaN,NaN,NaN


In [32]:
# Check how many rows of usable data we actually have
num_years = merged_microplastic_aqua_df.loc[
    merged_microplastic_aqua_df["Sampling Method"].eq("Neuston net"),
    "year"
].nunique()

print("Number of Years of Data: ", num_years)

# Print number of rows with Neustron net
num_rows = merged_microplastic_aqua_df["Sampling Method"].eq("Neuston net").sum()
print("Number of Neustron net rows: ", num_rows)
print()

# Print the number of rows (region, year) we can use for our regression
usable = (
    merged_microplastic_aqua_df[
        (merged_microplastic_aqua_df["Sampling Method"] == "Neuston net") &
        (merged_microplastic_aqua_df["metric_tons"].notna()) &
        (merged_microplastic_aqua_df["Microplastic_Mean"].notna())
    ][["assigned_region", "year", "metric_tons", "Microplastic_Mean", "Microplastic_Median", "Total_Samples"]]
    .drop_duplicates(subset=["assigned_region", "year"])   # 1 row per region-year
)
print("Usable regression rows (Neuston net, 1 per region-year):", len(usable))
print("Usable rows per region:\n", usable["assigned_region"].value_counts())
print()

# Print to ensure we have data for median, mean and sampling method
print("Number of rows", len(merged_microplastic_aqua_df))
merged_microplastic_aqua_df[merged_microplastic_aqua_df["year"] >= 1990].head()

Number of Years of Data:  27
Number of Neustron net rows:  190

Usable regression rows (Neuston net, 1 per region-year): 53
Usable rows per region:
 assigned_region
US_EAST_COAST     22
GULF_OF_MEXICO    18
US_WEST_COAST     13
Name: count, dtype: int64

Number of rows 625


,year,metric_tons,dollars,assigned_region,Sampling Method,Microplastic_Mean,Microplastic_Median,Total_Samples
49,1990,752850.0,667346642.0,GULF_OF_MEXICO,NaN,NaN,NaN,NaN
50,1990,74071.0,NaN,GULF_OF_MEXICO,NaN,NaN,NaN,NaN
51,1991,758539.0,681223248.0,GULF_OF_MEXICO,Neuston net,0.003548,0.00216,6.0
52,1991,97540.0,NaN,GULF_OF_MEXICO,Neuston net,0.003548,0.00216,6.0
53,1992,646832.0,655640926.0,GULF_OF_MEXICO,Neuston net,0.002268,0.00000,10.0


# Up to here, this is the correct dataset combined between aquaculture production and microplastic!!

Need to find another dataset for temperature

### Dataset 3: Temperature

In [43]:
ocean_df = pd.read_csv("./us-west-datasets/combined_temp_ph.csv")

# Clean column names
ocean_df.columns = [c.strip().lower().replace(" ", "_") for c in ocean_df.columns]

# Cast numeric columns where possible
for c in ["temperature", "salinity", "latitude", "longitude"]:
    if c in ocean_df.columns:
        ocean_df[c] = pd.to_numeric(ocean_df[c], errors="coerce")

# Extract Year and Month from UTC
ocean_df["date_utc"] = ocean_df["date_utc"].astype(str).str.strip().str.zfill(8)
ocean_df["year"] = ocean_df["date_utc"].str[:4].astype("Int64")
ocean_df["month"] = ocean_df["date_utc"].str[4:6].astype("Int64")

# Assign region using lat/lon
ocean_df["assigned_region"] = pd.Series([pd.NA] * len(ocean_df), dtype="string")
for region_name, b in REGIONS.items():
    mask = (
        ocean_df["latitude"].between(b["lat_min"], b["lat_max"]) &
        ocean_df["longitude"].between(b["lon_min"], b["lon_max"])
    )
    ocean_df.loc[mask, "assigned_region"] = region_name

# Keep only rows inside the 3 regions
print("Number of Rows before dropping rows not in regions: ", len(ocean_df))
ocean_df = ocean_df.dropna(subset=["assigned_region"]).copy()
print("Number of Rows after dropping rows not in regions: ", len(ocean_df))

# Optional: drop rows where BOTH temperature and ph are missing
ocean_df = ocean_df.dropna(subset=["temperature", "ph"], how="all")

print("Rows after region filter:", len(ocean_df))
print(ocean_df["assigned_region"].value_counts())

ocean_df.head(10)

Number of Rows before dropping rows not in regions:  159211
Number of Rows after dropping rows not in regions:  159211
Rows after region filter: 147234
assigned_region
US_WEST_COAST    147234
Name: count, dtype: Int64


,mooring_name,latitude,longitude,date_utc,time_utc,temperature,temp_flag,salinity,sal_flag,pco2_water,pco2_water_flag,dissolved_oxygen,dissolved_oxygen_flag,ph,ph_flag,source_file,year,month,assigned_region
0,Tillamook Bay,45.5538,-123.9159,20170803,0:45,12.41,NaN,31.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST
1,Tillamook Bay,45.5538,-123.9159,20170803,1:00,12.23,NaN,31.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST
2,Tillamook Bay,45.5538,-123.9159,20170803,1:15,11.71,NaN,31.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST
3,Tillamook Bay,45.5538,-123.9159,20170803,1:30,11.68,NaN,31.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST
4,Tillamook Bay,45.5538,-123.9159,20170803,1:45,11.40,NaN,32.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST
5,Tillamook Bay,45.5538,-123.9159,20170803,2:00,11.51,NaN,32.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST
6,Tillamook Bay,45.5538,-123.9159,20170803,2:15,11.09,NaN,32.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST
7,Tillamook Bay,45.5538,-123.9159,20170803,2:30,10.94,NaN,32.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST
8,Tillamook Bay,45.5538,-123.9159,20170803,2:45,10.87,NaN,32.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST
9,Tillamook Bay,45.5538,-123.9159,20170803,3:00,10.91,NaN,32.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv,2017,8,US_WEST_COAST


In [44]:
# Aggregate to determine median and mean temp based on (region, year)
ocean_df = (
    ocean_df.dropna(subset=["year", "assigned_region"])
    .groupby(["assigned_region", "year"], as_index=False)
    .agg(
        temp_mean=("temperature", "mean"),
        temp_median=("temperature", "median"),
        temp_samples=("temperature", "count"),
    )
    .sort_values(["assigned_region", "year"])
)

ocean_df.head(10)

,assigned_region,year,temp_mean,temp_median,temp_samples
0,US_WEST_COAST,2016,15.281775,14.980,3702
1,US_WEST_COAST,2017,12.349466,12.170,18193
2,US_WEST_COAST,2018,12.301564,11.910,53336
3,US_WEST_COAST,2019,13.726348,13.750,54517
4,US_WEST_COAST,2020,10.822796,9.955,14067


In [45]:
print(len(ocean_df))

5


In [ ]:
# Filter to keep useful cols
useful_cols = [
    "latitude",
    "longitude",
    "temperature",
    "salinity",
    "dissolved_oxygen",
    "ph",
    "year",
    "month",
    "assigned_region",
]
cols_keep = [c for c in useful_cols if c in ocean_df.columns]

# Create a copy
cleaned_ocean_df = ocean_df[cols_keep].copy()

# (Optional) Cast the remaining numeric columns too
for c in ["dissolved_oxygen", "ph"]:
    if c in cleaned_ocean_df.columns:
        cleaned_ocean_df[c] = pd.to_numeric(cleaned_ocean_df[c], errors="coerce")

cleaned_ocean_df.head(10)


,latitude,longitude,temperature,salinity,dissolved_oxygen,ph,year,month,assigned_region
0,45.5538,-123.9159,12.41,31.65,NaN,NaN,2017,8,US_WEST_COAST
1,45.5538,-123.9159,12.23,31.55,NaN,NaN,2017,8,US_WEST_COAST
2,45.5538,-123.9159,11.71,31.76,NaN,NaN,2017,8,US_WEST_COAST
3,45.5538,-123.9159,11.68,31.87,NaN,NaN,2017,8,US_WEST_COAST
4,45.5538,-123.9159,11.40,32.08,NaN,NaN,2017,8,US_WEST_COAST
5,45.5538,-123.9159,11.51,32.06,NaN,NaN,2017,8,US_WEST_COAST
6,45.5538,-123.9159,11.09,32.21,NaN,NaN,2017,8,US_WEST_COAST
7,45.5538,-123.9159,10.94,32.27,NaN,NaN,2017,8,US_WEST_COAST
8,45.5538,-123.9159,10.87,32.30,NaN,NaN,2017,8,US_WEST_COAST
9,45.5538,-123.9159,10.91,32.30,NaN,NaN,2017,8,US_WEST_COAST


In [ ]:
ocean_df = pd.read_csv("./datasets/combined_temp_ph_cleaned_annual_by_region.csv")

# Rename cols
ocean_df.columns = [c.strip().lower().replace(" ", "_") for c in ocean_df.columns]

# Cast col to numeric where necessary
ocean_df["year"] = pd.to_numeric(ocean_df["year"], errors="coerce").astype("Int64")
numeric_cols = [
    "temperature", "salinity", "latitude", "longitude",
]
for c in numeric_cols:
    if c in ocean_df.columns:
        ocean_df[c] = pd.to_numeric(ocean_df[c], errors="coerce")

# Assign a region to it
ocean_df["assigned_region"] = ocean_df["assigned_region"].astype("string")


# merged_all3 = aquaculture_df.merge(
#     ocean_df,
#     on=["year", "assigned_region"],
#     how="left"
# )

FileNotFoundError: [Errno 2] No such file or directory: './datasets/combined_temp_ph_cleaned_annual_by_region.csv'

In [37]:
ocean_df.head()

,MOORING_NAME,LATITUDE,LONGITUDE,DATE_UTC,TIME_UTC,TEMPERATURE,TEMP_FLAG,SALINITY,SAL_FLAG,pCO2_WATER,pCO2_WATER_FLAG,DISSOLVED OXYGEN,DISSOLVED OXYGEN_FLAG,PH,PH_FLAG,Source_File
0,Tillamook Bay,45.5538,-123.9159,20170803,0:45,12.41,NaN,31.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv
1,Tillamook Bay,45.5538,-123.9159,20170803,1:00,12.23,NaN,31.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv
2,Tillamook Bay,45.5538,-123.9159,20170803,1:15,11.71,NaN,31.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv
3,Tillamook Bay,45.5538,-123.9159,20170803,1:30,11.68,NaN,31.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv
4,Tillamook Bay,45.5538,-123.9159,20170803,1:45,11.40,NaN,32.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5_TILLAMOOK_BAY_Data.csv


In [ ]:
aquaculture_df = pd.read_csv("./datasets/global_production_quantity.csv")

# All should already be in Tonnes, but filtre just in case
aquaculture_df = aquaculture_df[aquaculture_df["Unit Name"] == "Tonnes - live weight"].copy()
# Rename cols
aquaculture_df = aquaculture_df.rename(columns={
    "Country Name En": "Country",
    "Global production by production source Quantity (1950 - 2023) Value value": "Aquaculture_Production_Tonnes",
})

# Cast cols to numeric where necessary
aquaculture_df["Year"] = pd.to_numeric(aquaculture_df["Year"], errors="coerce").astype("Int64")
aquaculture_df["Aquaculture_Production_Tonnes"] = pd.to_numeric(aquaculture_df["Aquaculture_Production_Tonnes"], errors="coerce")

# Filter for USA only and drop missing
us_aquaculture_df = aquaculture_df[aquaculture_df["Country"] == "United States of America"].copy()
us_aquaculture_df = us_aquaculture_df.dropna(subset=["Year", "Aquaculture_Production_Tonnes"])

# Sort by year
us_aquaculture_df = us_aquaculture_df.sort_values("Year").reset_index(drop=True)

print("Number of Rows:", len(us_aquaculture_df))
print("Year Range:", us_aquaculture_df["Year"].min(), "-", us_aquaculture_df["Year"].max())
print(us_aquaculture_df.head())

**Insights**
1. Presently, the only aquaculture production dataset we found is for country total for USA.
2. Our microplastics is divided into 3 regions presently
- 

In [ ]:
# First, combine all regions in the microplastic into 1 US, since asuming only 1 sampling method remains per year
micro_national_df = (
    cleaned_us_all_regions_df
    # Group by Year and Sampling Method still
    .groupby(["Year", "Sampling Method"], as_index=False)
    .agg(
        Microplastic_Mean=("Microplastic_Mean", "mean"),
        Microplastic_Median=("Microplastic_Median", "median"),
        Total_Samples=("Total_Samples", "sum")
    )
    .sort_values("Year")
)

print(micro_national_df.groupby("Year")["Sampling Method"].nunique().max())

Therefore, we must collapse sampling methods too

In [ ]:
# Group by Year only since 1 year may have 3 sampling methods
microplastic_national_df = (
    cleaned_us_all_regions_df
    .groupby("Year", as_index=False)
    .agg(
        Microplastic_Mean=("Microplastic_Mean", "mean"),
        Microplastic_Median=("Microplastic_Median", "median"),
        Total_Samples=("Total_Samples", "size")
    )
    .sort_values("Year")
)
microplastic_national_df.head()

In [ ]:
# Ensure Year are numeric
microplastic_national_df["Year"] = microplastic_national_df["Year"].astype("Int64")
us_aquaculture_df["Year"] = us_aquaculture_df["Year"].astype("Int64")

# Merge the 2 Datasets first, keep only overlapping years with inner merge
merged_microplastic_aquaculture_df = microplastic_national_df.merge(
    us_aquaculture_df,
    on="Year",
    how="inner"
)

print("2 Dataset Merge size:", len(merged_microplastic_aquaculture_df))
print("Year range:", merged_microplastic_aquaculture_df["Year"].min(), "-", merged_microplastic_aquaculture_df["Year"].max())
print(merged_microplastic_aquaculture_df.head())

### 1.3 Dataset 3 - Temperature, pH etc (New Hypo testing basically; INCOMPLETE)

Since the dataset dont match nicely, we should do multiple different hypothesis testing to arrive at our answer. E.g. Aquaculture with Temp, Aquaculture with Microplastic etc

I cannot fit this with the previous aquaculture, so I used traditional fishing as a proxy since it affects how much fishes can multiply and grow

#### This is not aquaculture, but commercial fishing. Dk if we can use it as a substitute for now

In [ ]:
# Import data
aquaculture_prod_df = pd.read_csv("./datasets/us_three_regions_fish_production_annual.csv")

# Rename columns for those that need
aquaculture_prod_df = aquaculture_prod_df.rename(columns={
    "Region Name": "NMFS_Region",
    "Metric Tons": "Metric_Tons",
})
aquaculture_prod_df.head()

In [ ]:
# Remove comma from numbers
for col in ["Pounds", "Dollars", "Metric_Tons"]:
    if col in aquaculture_prod_df.columns:
        aquaculture_prod_df[col] = pd.to_numeric(
            aquaculture_prod_df[col].astype(str).str.replace(",", "", regex=False).str.strip(),
            errors="coerce"
        )

# Cast columns to numeric where necesssary
aquaculture_prod_df["Year"] = pd.to_numeric(aquaculture_prod_df["Year"], errors="coerce").astype("Int64")
aquaculture_prod_df["Pounds"] = pd.to_numeric(aquaculture_prod_df["Pounds"], errors="coerce").astype("Int64")
aquaculture_prod_df["Dollars"] = pd.to_numeric(aquaculture_prod_df["Dollars"], errors="coerce").astype("Int64")
aquaculture_prod_df["Metric_Tons"] = pd.to_numeric(aquaculture_prod_df["Metric_Tons"], errors="coerce").astype("Int64")

# Drop missing data for year, region, tons
aquaculture_prod_df = aquaculture_prod_df.dropna(subset=["Year", "NMFS_Region", "Metric_Tons"])

# filter commercial only for now, no need recreational
aquaculture_prod_df = aquaculture_prod_df[aquaculture_prod_df["Collection"].astype(str).str.strip().str.lower() == "commercial"].copy()

# Print results first to see if we need further cleaning
print("Number of Rows after Basic Cleaning:", len(aquaculture_prod_df))
print("Regions Found:", sorted(aquaculture_prod_df["NMFS_Region"].unique())[:20])
print("Year Range:", int(aquaculture_prod_df["Year"].min()), "to", int(aquaculture_prod_df["Year"].max()))

In [ ]:
# Match the different NMFS regions to the 3 US EAST, WEST, and GULF of MEXICO I found
US_EAST_NMFS_REGIONS = {"New England", "Middle Atlantic", "South Atlantic"}

# Create a new Region Col
aquaculture_prod_df["Region"] = None
aquaculture_prod_df.head()
aquaculture_prod_df.loc[aquaculture_prod_df["NMFS_Region"].eq("Gulf"), "Region"] = "GULF_OF_MEXICO"
aquaculture_prod_df.loc[aquaculture_prod_df["NMFS_Region"].eq("Pacific Coast"), "Region"] = "US_WEST_COAST"
aquaculture_prod_df.loc[aquaculture_prod_df["NMFS_Region"].isin(US_EAST_NMFS_REGIONS), "Region"] = "US_EAST_COAST"

# Drop any rows without Region
cleaned_aquaculture_prod_df = aquaculture_prod_df.dropna(subset=["Region"]).copy()

# Aggregate the total production annually for each region
us_region_aquaculture_df = (
    # Group by Region and Year
    cleaned_aquaculture_prod_df.groupby(["Region", "Year"], as_index=False)
    # Aggregate by taking sum of tons, dollars and pounds too ig
    .agg(
        Fish_Metric_Tons=("Metric_Tons", "sum"),
        Fish_Pounds=("Pounds", "sum"),
        Fish_Dollars=("Dollars", "sum"),
    )
    .sort_values(["Region", "Year"])
)

print("\nUS Regional Annual Aquaculture:")
print("Number of Rows:", len(us_region_aquaculture_df))
print(us_region_aquaculture_df.head())

In [ ]:
# Read dataset 3
ocean_df = pd.read_csv("./datasets/realistic_ocean_climate_dataset.csv")

ocean_df.head()

In [ ]:
# Cast Year and assign region
def assign_region(lat, lon):
    if 18 <= lat <= 31 and -98 <= lon <= -80:
        return "GULF_OF_MEXICO"
    elif 32 <= lat <= 49 and -130 <= lon <= -117:
        return "US_WEST_COAST"
    elif 25 <= lat <= 47 and -82 <= lon <= -66:
        return "US_EAST_COAST"
    else:
        return None

ocean_df["Region"] = None
ocean_df["Region"] = ocean_df.apply(
    lambda row: assign_region(row["Latitude"], row["Longitude"]),
    axis=1
)

ocean_df = ocean_df.dropna(subset=["Region"])
ocean_df["Year"] = pd.to_datetime(ocean_df["Date"]).dt.year
ocean_df.head()

## 2 EDA for Microplastic and Aquaculture Production

In [ ]:
print(cleaned_us_all_regions_df.groupby(["Sampling Method"])["Microplastic_Mean"].mean())
print(cleaned_us_all_regions_df.groupby(["Sampling Method"])["Year"].mean())

In [ ]:
# First, we will combine the 2 datasets where sampling method was not collapsed
micro_regional_df = (
    cleaned_us_all_regions_df
    .groupby(["Region", "Year", "Sampling Method"], as_index=False)
    .agg(
        Microplastic_Mean=("Microplastic_Mean", "mean"),
        Microplastic_Median=("Microplastic_Median", "median"),
        Total_Samples=("Total_Samples", "sum")
    )
    .sort_values(["Region", "Year", "Sampling Method"])
)

print("Rows:", len(micro_regional_df))
print(micro_regional_df.head())

In [ ]:
# Remove the only factors that matters from aquaculture
us_aqua_small = us_aquaculture_df[[
    "Year",
    "Aquaculture_Production_Tonnes"
]].copy()

# Ensure year is numeric
micro_regional_df["Year"] = micro_regional_df["Year"].astype("Int64")
us_aqua_small["Year"] = us_aqua_small["Year"].astype("Int64")

merged_df = micro_regional_df.merge(
    us_aqua_small,
    on="Year",
    how="inner"
)

print("Merged size:", len(merged_df))
print("Year range:",
      merged_df["Year"].min(),
      "-",
      merged_df["Year"].max())

print(merged_df.head())

In [ ]:
# Separate the dataset by sampling methods since microplastic conc is affected
print(merged_df["Sampling Method"].value_counts())

In [ ]:
# Filter out the most common one
neuston_df = merged_df[merged_df["Sampling Method"] == "Neuston net"].copy()
neuston_df.head()

In [ ]:
# Check shape of dataset
print("Rows:", len(neuston_df))
print("Year range:", neuston_df["Year"].min(), "-", neuston_df["Year"].max())
print("Regions:", neuston_df["Region"].unique())

In [ ]:
# Summar Stats
print(neuston_df[[
    "Microplastic_Mean",
    "Microplastic_Median",
    "Aquaculture_Production_Tonnes",
    "Total_Samples"
]].describe())

In [ ]:
# Missing Values Checl
print(neuston_df.isnull().sum())

In [ ]:
# Check for zero mean and medians for Log later
print("Zero Microplastic Mean:", (neuston_df["Microplastic_Mean"] == 0).sum())

print("Zero Microplastic Median:",(neuston_df["Microplastic_Median"] == 0).sum())

In [ ]:
# Regional Comparison of Microplastics
print(neuston_df.groupby("Region")[[
        "Microplastic_Mean",
        "Aquaculture_Production_Tonnes"
    ]].mean())

In [ ]:
# Distribution Plot of Mean
plt.hist(neuston_df["Microplastic_Mean"], bins=15)
plt.title("Distribution of Microplastic Mean (Neuston)")
plt.show()

In [ ]:
# Distribution Plot of Median
plt.hist(neuston_df["Microplastic_Median"], bins=15)
plt.title("Distribution of Microplastic_Median (Neuston)")
plt.show()

In [ ]:
for region in neuston_df["Region"].unique():
    subset = neuston_df[neuston_df["Region"] == region]
    plt.plot(subset["Year"], subset["Microplastic_Mean"], label=region)

plt.legend()
plt.title("Microplastic Trend by Region (Neuston)")
plt.show()

In [ ]:
# Aquaculture production by region
for region in neuston_df["Region"].unique():
    subset = neuston_df[neuston_df["Region"] == region]
    plt.plot(subset["Year"], subset["Aquaculture_Production_Tonnes"], label=region)

plt.legend()
plt.title("Aquaculture Trends by Region (National Repeated)")
plt.show()

In [ ]:
# Scatter plots by region
for region in neuston_df["Region"].unique():
    subset = neuston_df[neuston_df["Region"] == region]
    plt.scatter(
        subset["Microplastic_Mean"],
        subset["Aquaculture_Production_Tonnes"],
        label=region
    )

plt.legend()
plt.xlabel("Microplastic Mean")
plt.ylabel("Aquaculture Production")
plt.title("Microplastic vs Aquaculture (by Region)")
plt.show()

In [ ]:
# Check for Outliers for Mean
print(neuston_df.sort_values("Microplastic_Mean", ascending=False).head(10))

In [ ]:
# Check for Outliers for Median
print(merged_df.sort_values("Microplastic_Median", ascending=False).head(10))

In [ ]:
# Variance of Mean over Time and Region
print("Overall variance:", neuston_df["Microplastic_Mean"].var())
print("Between-region variance:", neuston_df.groupby("Region")["Microplastic_Mean"].mean().var())

In [ ]:
# Total variance
total_var = neuston_df["Microplastic_Median"].var()

# Between-region variance (using mean across time)
between_var = (
    neuston_df
    .groupby("Region")["Microplastic_Median"]
    .mean()
    .var()
)

# Within-region variance
within_var = (
    neuston_df
    .groupby("Region")["Microplastic_Median"]
    .var()
    .mean()
)

print("Total variance:", total_var)
print("Between-region variance:", between_var)
print("Within-region variance:", within_var)

In [ ]:
# Since we know there are 0 mean and medians, we inspect smallest
min_positive = neuston_df.loc[
    neuston_df["Microplastic_Mean"] > 0,
    "Microplastic_Mean"
].min()

print("Smallest positive microplastic mean:", min_positive)

In [ ]:
# Use half of smallest
c = min_positive / 2
print("Chosen constant:", c)

neuston_df["log_microplastic_mean"] = np.log(neuston_df["Microplastic_Mean"] + c)
neuston_df["log_microplastic_median"] = np.log(neuston_df["Microplastic_Median"] + c)
neuston_df["log_aquaculture"] = np.log(neuston_df["Aquaculture_Production_Tonnes"])

for region in neuston_df["Region"].unique():
    subset = neuston_df[neuston_df["Region"] == region]
    plt.scatter(subset["log_microplastic_mean"], subset["log_aquaculture"], label=region)

plt.legend()
plt.title("Log-Log Relationship by Region (Mean with Aqua Production)")
plt.show()

# Plot for median now
for region in neuston_df["Region"].unique():
    subset = neuston_df[neuston_df["Region"] == region]
    plt.scatter(subset["log_microplastic_median"], subset["log_aquaculture"], label=region)

plt.legend()
plt.title("Log-Log Relationship by Region (Median with Aqua Production)")
plt.show()

In [ ]:
# Correlation MAtrix
print(neuston_df[[
    "Microplastic_Mean",
    "Aquaculture_Production_Tonnes"
]].corr())

print(neuston_df[[
    "log_microplastic_mean",
    "log_aquaculture"
]].corr())

## 3 Regression

In [ ]:
from sklearn.linear_model import LinearRegression

# Simple log linear model
X = neuston_df[["log_microplastic_mean"]]
y = neuston_df["log_aquaculture"]

model = LinearRegression()
model.fit(X, y)

print("Intercept:", model.intercept_)
print("Gradient:", model.coef_[0])
print("R-square:", model.score(X, y))

In [ ]:
from sklearn.linear_model import LinearRegression

# Create region dummies
X = pd.get_dummies(
    neuston_df[["log_microplastic_mean", "Region"]],
    drop_first=True
)
y = neuston_df["log_aquaculture"]

model = LinearRegression()
model.fit(X, y)

print("Intercept:", model.intercept_)
print("Coefficients:", dict(zip(X.columns, model.coef_)))
print("R-square:", model.score(X, y))

**Insights**
- After viewing my datasets, I think it will be ideal to combine

In [ ]:
# # Moving to Dataset 2
# aquaculture_prod_df = pd.read_csv("./datasets/global_production_quantity.csv")

# # Ensure Year and production is numeric
# aquaculture_prod_df["Year"] = pd.to_numeric(aquaculture_prod_df["Year"], errors="coerce")

# aquaculture_prod_df["Global production by production source Quantity (1950 - 2023) Value value"] = pd.to_numeric(
#     aquaculture_prod_df["Global production by production source Quantity (1950 - 2023) Value value"],
#     errors="coerce"
# )

# # Keep the following 4 cols and rename them if needed
# aquaculture_prod_df = aquaculture_prod_df[[
#     "Country Name En",
#     "Year",
#     "Global production by production source Quantity (1950 - 2023) Value value",
#     "Global production by production source Quantity (1950 - 2023) Value flag"
# ]].rename(columns={
#     "Country Name En": "Country",
#     "Global production by production source Quantity (1950 - 2023) Value value": "Aquaculture Production (Tonnes)",
#     "Global production by production source Quantity (1950 - 2023) Value flag": "Production Flag"
# })

# print("Number of Rows:", len(aquaculture_prod_df))
# print(aquaculture_prod_df["Year"].unique())

# test = aquaculture_prod_df[
#     aquaculture_prod_df["Country"] == "United States of America"
# ]

# print("Number of Rows:", len(test))
# test.head()
